# Run PESTPP-OPT

In this notebook we will setup and solve a mgmt optimization problem around how much groundwater can be pumped while maintaining sw-gw exchange

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu
%matplotlib inline

## SUPER IMPORTANT: SET HOW MANY PARALLEL WORKERS TO USE

In [ ]:
num_workers = 20

In [ ]:
t_d = "template_history"
m_d = "master_opt"

### We can look at the summary information about the parameters

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
pst.write_par_summary_table(filename="none").sort_index()

### define our decision variable group and also set some `++args`.

Conceptually, we are going to optimize current pumping rates to make sure we meet ecological flows under both historic (current) conditions and scenario (future) conditions.  Remember the scenario is an extreme 1-year drought, so if we pump too much now, the system will be too low to provide critical flows if next year is an extreme drough - transient memory!

Define a parameter group as the devision variables (i.e. the variables that we will tune to meet the optimal condition). We will define `wellflux_k02` as the decision variable group (defined by the `++arg` called `opt_dec_var_groups`. Note in the table above that this group represents (time-invariant) flux mulipliers for each of the 6 wells (we will however only optimize for pumping rates in current (historic) conditions). 

We can also define which direction we want the optimization to go using `opt_direction` as `max`. This means the objective of the optimization will be to maximize future pumping subject to the constraints we will establish below.


In [ ]:
pst.pestpp_options = {}
#dvg = ["welflux_k02","welflux"]
dvg = ["welflux_k02"]  # time-invariant flux multiplier for each well
pst.pestpp_options["opt_dec_var_groups"] = dvg
pst.pestpp_options["opt_direction"] = "max"

For the first run, we won't use chance constraints, so just fix all non-decision-variable "parameters".  We also need to set some realistic bounds on the `welflux` multiplier decision variables.  Finally, we need to specify a larger derivative increment for the decision varible group. For typical parameter estimation, `derinc=0.01` is often sufficient for calculating a Jacobian matrix. But, for the response matrix method of optimization, the response can be subtle requiring a greater perturbation increment. We will set it to `0.25` using some `pandas` manipulation.

In [ ]:
par = pst.parameter_data
par.loc[:,"partrans"] = "fixed"

temporal_wel_pars = par.loc[par.parnme.apply(lambda x: "welflux" in x),"parnme"]
#print(temporal_wel_pars)
#par.loc[temporal_wel_pars,"parlbnd"] = 0.0 
par.loc[temporal_wel_pars,"parval1"] = 1.0

# # dec vars
dvg_pars = par.loc[par.pargp.apply(lambda x: x in dvg),"parnme"]
par.loc[dvg_pars,"partrans"] = "none"
par.loc[dvg_pars,"parlbnd"] = 0.0
par.loc[dvg_pars,"parubnd"] = 3.0  # corresponds to -450 m3/d
par.loc[dvg_pars,"parval1"] = 1.0

par.loc[dvg_pars,:]

In [ ]:
pst.rectify_pgroups()
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"inctyp"] = "absolute"
pst.parameter_groups.loc[dvg,"derinc"] = 0.25

pst.parameter_groups.loc[dvg,:]

### define constraints

Model-based or dec var-related constraints are identified in `pestpp-opt` by an obs or prior information group that starts with "less_than" or "greater_than" and a weight greater than zero.  So first, we turn off all of the weights and get names for the sw-gw exchange forecasts (funny how optimization turns forecasts into constraints...)

In [ ]:
obs = pst.observation_data
obs.loc[:,"weight"] = 0.0
swgw_const = obs.loc[obs.obsnme.apply(lambda x: "fa" in x and ("hw" in x or "tw" in x)),"obsnme"]
obs.loc[swgw_const,:].obsval.hist(bins=20)

We need to change the obs group (`obgnme`) so that `pestpp-opt` will recognize these model outputs as constraints.  The `obsval` becomes the RHS of the constraint.  

In [ ]:
obs.loc[swgw_const,"obgnme"] = "less_than"
obs.loc[swgw_const,"weight"] = 1.0

# we must have at least 300 m3/day of flux from gw to sw
# for historic and scenario periods
# and for both headwaters and tailwaters
obs.loc[swgw_const,"obsval"] = -100

Now we need to define a minimum total pumping rate, otherwise this opt problem might yield a solution that doesn't give enough water for the intended usage.  We will do this through a prior information constraint since this just a sum of decision variable values - the required minimum value will the sum of current pumping rates:

In [ ]:
pyemu.pst_utils.pst_config["prior_fieldnames"]

Since all pumping wells are using the same rate and are of same "water supply benefit", we can just use a `1.0` multiplier in front of each `wel.flux` decision varialbe.  If that is not the case, then you need to set the multipliers to be more meaningful

In [ ]:
pi = pst.null_prior
pi.loc["pi_1","obgnme"] = "greater_than"
pi.loc["pi_1","pilbl"] = "pi_1"
pi.loc["pi_1","equation"] = " + ".join(["1.0 * {0}".format(d) for d in dvg_pars]) +\
                            " = {0}".format(par.loc[dvg_pars,"parval1"].sum())
pi.loc["pi_1","weight"] = 1.0
pi.equation["pi_1"]

In [ ]:
pst.prior_information

## now for a risk-neutral optimization (ignoring uncertainty in constraints)

#### Note that setting `noptmax=1` is equivalent to selecting Linear Programming (LP) as the optimization algorithm (thus assuming a linear response matrix).

#### A higher value of `noptmax` runs Sequential Linear Programming (SLP)

In [ ]:
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt.pst"))
pyemu.os_utils.start_workers(t_d,"pestpp-opt","freyberg_opt.pst",num_workers=num_workers,master_dir=m_d)

Let's load and inspect the response matrix

In [ ]:
jco = pyemu.Jco.from_binary(os.path.join(m_d,"freyberg_opt.1.jcb")).to_dataframe().loc[pst.less_than_obs_constraints,:]
jco

We see the transient effects in the nonzero value between current pumping rates (columns) and scenario sw-gw exchange (rows from 1980) and that the current 1979 sw-gw exchanges are always larger than those in the scenario 1980 condition.

Let's also load the optimal decision variable values:

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

The sum of these values is the optimal objective function value. However, since these are just mulitpliers on the pumping rate, this number isnt too meaningful. Instead, lets look at the residuals file

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt.pst"),resfile=os.path.join(m_d,"freyberg_opt.1.sim.rei"))
pst.res.loc[pst.nnz_obs_names,:].sort_values(by="modelled",ascending=False).modelled.iloc[:10].plot(kind="bar")

we can see the "binding constraints" as those at -100

We better also check that our prior information constraint (min total abstracted water allocation) is being met..

In [ ]:
pst.res.loc[pst.prior_names,:]

### Opt under uncertainty part 1: FOSM chance constraints

This is where the process of uncertainty quantification/history matching and mgmt optimizatiom meet - worlds collide! 

Mechanically, in PESTPP-OPT, to activate the chance constraint process, we need to specify a `risk != 0.5`.  Risk ranges from 0.001 (risk tolerant) to 0.999 (risk averse).  The larger the risk value, the more confidence we have that the (uncertain) model-based constraints are truely satisfied.  Here we will start with a risk tolerant stance:

In [ ]:
pst.pestpp_options["opt_risk"] = 0.4

For the FOSM-based chance constraints, we also need to have at least one adjustable non-dec-var parameter so that we can propogate parameter uncertainty to model-based constraints (this can also be posterior FOSM is non-constraint, non-zero-weight observations are specified).  For this simple demo, lets just use the constant multiplier parameters in the prior uncertainty stance:

In [ ]:
cn_pars = par.loc[par.pargp.apply(lambda x: "cn" in x),"parnme"]
cn_pars

In [ ]:
par = pst.parameter_data
par.loc[cn_pars,"partrans"] = "log"
pst.control_data.noptmax = 1
pst.write(os.path.join(t_d,"freyberg_opt_uu1.pst"))

So now we need to not only fill the response matrix (between dec vars and constraints) but we also need to fill the jacobian matrix (between parameters and constraints).  Given that we only have 6 decision variables, let's just re-populate the response matrix while also populating the Jacobian.

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-opt","freyberg_opt_uu1.pst",num_workers=num_workers,master_dir=m_d)

What do we expect to see here?  What should happen to the optimal dec vars? And the constraints?

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt_uu1.pst"),resfile=os.path.join(m_d,"freyberg_opt_uu1.1.sim.rei"))
pst.res.loc[pst.nnz_obs_names,:].sort_values(by="modelled",ascending=False).modelled.iloc[:20].plot(kind="bar")

Wait - was the constraint that sw-gw flux had to be less than -100?  How do we have some many constraints greater than -100???

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu1.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

We now see how taking a risk tolerant stance allows for more pumping but that we have only a 40% chance of actually satifying the sw-gw constraints (see how the model simulated value is actually in violation of the -100 constraint RHS).  Lets check the residuals that include the FOSM-based chance constraint shift:

In [ ]:
res_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_uu1.1.sim+fosm.rei")).loc[pst.nnz_obs_names,:]
res_df.sort_values(by="modelled",ascending=False).modelled.iloc[:10].plot(kind="bar")

See what is happening?  Because we were risk tolerant, we actually subtracted some sw-gw flux from the values from MODFLOW - effectively acounting for additional sw-gw that wasnt acutally simulated.  This flux is in the uncertainty in the simulated sw-gw flux.  And since we were "aggresive" with our risk-stance, we can exploit this uncertainty to extract additional groundwater.

### Opt under uncertainty part 2: ensemble-based chance constraints

PESTPP-OPT can also skip the FOSM calculations if users specify model-based constraint weights as standard deviations (i.e. uncertainty in the forecasts/constraints).  These can be derived from our existing ensembles (oh snap!)

In [ ]:
obs_df = pd.read_csv(os.path.join("master_prior_sweep","sweep_out.csv"),index_col=0)
obs_df = obs_df.loc[obs_df.failed_flag==0,:]

In [ ]:
pr_std = obs_df.std().loc[pst.nnz_obs_names]
pr_std

Note we can also skip the FOSM calcs within `PESTPP-OPT` using weights as per previous FOSM standard deviation calcs, not just ensemble-based standard deviations

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,"weight"] = pr_std.loc[pst.nnz_obs_names]
pst.pestpp_options["opt_std_weights"] = True
pst.write(os.path.join(t_d,"freyberg_opt_uu2.pst"))

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-opt","freyberg_opt_uu2.pst",num_workers=num_workers,master_dir=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_uu2.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

Why is the objective function higher when we use the ensemble-based constraint uncertainty compared to the FOSM constraint uncertainty?  Remember how many more parameters were used in the ensemble analyses compared to just the hand full of constant by layer parameters that we used for the FOSM calcs within PESTPP-OPT?

### Super secret mode for `LP`

It turns out, if the opt problem is truely linear, we can reuse results of a previous PESTPP-OPT run to modify lots of the pieces of the optimization problem and resolve the optimization problem without running the model even once!  WAT!? 

As long as the same decision variables are relates to the same responses, and we can fairly assume that the response matrix that relates the decision variables to the constraints is linear, then the response matrix doesn't change even if things like bounds and risk level change. We just need `pestpp-opt` to read in the response matrix (which is stored with the same format as a Jacobian (`jcb`)) and the residuals (`rei`). 

This is done by specifying some additional `++args` (and copying some files around)

In [ ]:
shutil.copy2(os.path.join(m_d,"freyberg_opt_uu2.1.jcb"),os.path.join(m_d,"restart.jcb"))
shutil.copy2(os.path.join(m_d,"freyberg_opt_uu2.1.jcb.rei"),os.path.join(m_d,"restart.rei"))

Once we have copied over the necessary files, we set a few `++args`:  
* `base_jacobian`: this instructs `pestpp-opt` to read in the existing response matrix
* `hotstart_resfile`: this instructs `pestpp-opt` to use the residuals we already have
* `opt_skip_final`: this waives the usual practice of running the model once with optimal parameter values

Which runs do each of these skip specifically?

In [ ]:
pst.pestpp_options["base_jacobian"] = "restart.jcb"
pst.pestpp_options["hotstart_resfile"] = "restart.rei"
pst.pestpp_options["opt_skip_final"] = True
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))

In [ ]:
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

Oh snap!  that means we can do all sort of kewl optimization testing really really fast...

## now let's try taking a risk averse stance

In [ ]:
risk = 0.55

In [ ]:
pst.pestpp_options["opt_risk"] = risk
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

### now lets evaluate how our OUU problem changes if we use posterior standard deviations - this is a critically important use of the uncertainty analysis from history matching...

This is where all that painful history-matching finally has a real-world (and valid) purpose: to reduce the uncertainty in the constraints for a decision-support optimization problem

In [ ]:
obs_df = pd.read_csv(os.path.join("master_ies","freyberg_ies.3.obs.csv"),index_col=0)

#df = df=pd.read_csv(os.path.join("master_glm","freyberg_pp.post.obsen.csv"),index_col=0)
#obs_df = pyemu.ObservationEnsemble.from_dataframe(pst=pst,df=df)
#obs_df = obs_df.loc[obs_df.phi_vector.sort_values().index[:20],:] 
pt_std = obs_df.std().loc[pst.nnz_obs_names]
obs_df.std().loc[pst.nnz_obs_names]
#obs_df.max().loc[pst.nnz_obs_names]

How much lower is the posterior standard deviations as compared to the prior?

In [ ]:
pd.DataFrame({"prior":pr_std,"posterior":pt_std}).plot(kind="bar",figsize=(20,5))

This implies that the chance constraints (which express the important model input uncertainty propogated to the forecast/constraints) is significantly lower, meaning uncertainty has less "value" in the optimization objective function

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,"weight"] = pt_std.loc[pst.nnz_obs_names]
pst.observation_data.loc[pst.nnz_obs_names,"weight"].plot(kind="hist")

In [ ]:
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)

In [ ]:
par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
print(par_df.loc[dvg_pars,"parval1"].sum())
par_df.loc[dvg_pars,:]

In [ ]:
df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_restart.1.est+fosm.rei")).loc[pst.nnz_obs_names,:]
df.sort_values(by="modelled",ascending=False).modelled.iloc[:10].plot(kind="bar")

Again we see that historic tail water flux is the binding constraint.  

## some reformulation of the opt problem. Can we open up decision variable (feasibility) space?

Lets reformulate the problem to be constrained by the total sw-gw flux across all reaches instead of splitting into headwaters and tailwaters.  Good thing we have added the list file budget components to the control file!

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"freyberg_opt_restart.pst"))
obs = pst.observation_data
obs.loc[pst.nnz_obs_names,"obgnme"] = "sw-gw"  # hw and tw constraints from tagged "less_than"
obs.loc[pst.nnz_obs_names,"weight"] = 0.0

In [ ]:
tot_swgw = obs.loc[obs.obsnme.str.startswith("flx_stream_"),"obsnme"]
tot_swgw

In [ ]:
obs.loc[tot_swgw,"obgnme"] = "less_than"
obs.loc[tot_swgw,"weight"] = 1.0
obs.loc[tot_swgw,"weight"] = obs_df.std().loc[pst.nnz_obs_names]
obs.loc[tot_swgw,"obsval"] = -200  # simple linear sum


Check your understanding:  we just reformulated the mgmt optimization problem to use the total sw-gw flux along all reaches instead of dividing the constraints into a headwater and tailwater reach grouping

In [ ]:
obs_df.std().loc[pst.nnz_obs_names].plot(kind="bar")

## risk sweeps

Since we really want to find the most risk averse stance that is still feasible we will run a sweep of risk values:

In [ ]:
par_dfs = []
res_dfs = []
risk_vals = np.arange(0.05,1.0,0.05)
for risk in risk_vals:
    #try:
    #    os.remove(os.path.join(m_d,"freyberg_opt_restart.1.est+fosm.rei"))
    #except:
    #    pass
   
    pst.pestpp_options["opt_risk"] = risk
    pst.pestpp_options["opt_skip_final"] = True
    print("undertaking evaluation for risk value: {0:.2f}".format(risk))
    pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
    pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)
    
    par_df = pyemu.pst_utils.read_parfile(os.path.join(m_d,"freyberg_opt_restart.1.par"))
    par_df = par_df.loc[dvg_pars,:]
    #when the solution is infeasible, pestpp-opt writes extreme negative values 
    # to the par file:
    if par_df.parval1.sum() < 6.0: 
        print("infeasible at risk {0:.2f}".format(risk))
        break
    
    res_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_restart.1.est+fosm.rei"))
    res_df = res_df.loc[pst.nnz_obs_names,:]
    res_dfs.append(res_df.modelled)
    par_dfs.append(par_df.parval1)

# process the dec var and constraint dataframes for plotting
risk_vals = risk_vals[:len(par_dfs)]
par_df = pd.concat(par_dfs,axis=1).T
par_df.index = risk_vals
par_df.index = par_df.index.map(lambda x: "{0:0.3f}".format(x))
res_df = pd.concat(res_dfs,axis=1).T
res_df.index = risk_vals
res_df.index = res_df.index.map(lambda x: "{0:0.3f}".format(x))

In [ ]:
colors = ["m","c","g","r","b","orange"]
fig, axes = plt.subplots(2,1,figsize=(20,15))
par_df.plot(kind="bar",ax=axes[0],alpha=0.75,color=colors).legend(bbox_to_anchor=(1.2, 0.5))
axes[0].set_ylabel("individual pumping rates")
axes[0].set_xticklabels([])
res_df.plot(kind="bar",ax=axes[1],alpha=0.75,edgecolor="none").legend(bbox_to_anchor=(1.2, 0.5))
axes[1].plot(axes[1].get_xlim(),[-200,-200],"r--",lw=3)
axes[1].set_ylabel("sw-gw flux")
axes[1].set_xlabel("risk");

Now for some maps of pumping regimes

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=t_d)

wf_par = pst.parameter_data.loc[dvg_pars,:].copy()
wf_par.loc[:,"k"] = wf_par.parnme.apply(lambda x: int(x[2:4]))
wf_par.loc[:,"i"] = wf_par.parnme.apply(lambda x: int(x[4:8]))
wf_par.loc[:,"j"] = wf_par.parnme.apply(lambda x: int(x[8:]))
wf_par.loc[:,"x"] = wf_par.apply(lambda x: m.sr.xcentergrid[x.i,x.j],axis=1)
wf_par.loc[:,"y"] = wf_par.apply(lambda x: m.sr.ycentergrid[x.i,x.j],axis=1)

ib = m.bas6.ibound[0].array
ib = np.ma.masked_where(ib!=0,ib)
fig,axes = plt.subplots(5,int(np.ceil(par_df.shape[0]/5)),figsize=(12,12))
axes = axes.flatten()
for risk,ax in zip(par_df.index,axes):
    ax.set_aspect("equal")
    #ax = plt.subplot(111,aspect="equal") 
    ax.imshow(ib,extent=m.sr.get_extent())
    ax.scatter(wf_par.x,wf_par.y,s=par_df.loc[risk,wf_par.parnme].values*50,c=colors)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(risk)
    
for i in range(par_df.shape[0],axes.shape[0]):
    ax = axes[i]
    ax.axis("off")
    

How slick was that!  no more model runs needed and yet we transformed the OUU problem (by swapping constraints) and solved for a much more risk averse stance!  Just to make sure, lets run the model with the most risk-averse decision variables:

In [ ]:
pst.pestpp_options["opt_risk"] = risk_vals[-1]
pst.pestpp_options["opt_skip_final"] = False
pst.write(os.path.join(m_d,"freyberg_opt_restart.pst"))
pyemu.os_utils.run("pestpp-opt freyberg_opt_restart.pst",cwd=m_d)
# load the simulated outputs plus the FOSM chance constraint offsets:
res_df = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_restart.1.sim+fosm.rei"))
res_df = res_df.loc[pst.nnz_obs_names,:]
res_df.sort_values(by="modelled",ascending=False).modelled.iloc[:10].plot(kind="bar")

In [ ]:
# load the actual model simulated outputs
res_df_sim = pyemu.pst_utils.read_resfile(os.path.join(m_d,"freyberg_opt_restart.1.sim.rei"))
res_df_sim = res_df_sim.loc[pst.nnz_obs_names,:]
ax = pd.DataFrame({"sim":res_df_sim.modelled,"sim+fosm":res_df.modelled}).plot(kind="bar",figsize=(20,10))
ax.plot(ax.get_xlim(),[-200,-200],"r--",lw=3)

Here we can see the cost of uncertainty - we have to simulate a greater flux from gw to sw to make sure (e.g. be risk averse) that the flux from  gw to sw is actually at least 200 m3/day


# FINALLY!!!

We now see the reason for high-dimensional uncertainty quantification and history matching: to define and then reduce (through data assimilation) the uncertainty in the model-based constraints (e.g. sw-gw forecasts) so that we can find a more risk-averse management solution - we can use a model to identify an optimal pumping scheme to provide the volume of water needed for water supply, agriculture, etc. but also provide assurances (at the given confidence) that ecological flows will be maintained under both current conditions and in the event of an extreme 1-year drought.  BOOM!